In [ ]:
import os
import numpy as np
import cv2  # Import OpenCV for resizing
import numpy as np
import matplotlib.pyplot as plt

import librosa
import librosa.display

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder

print(tf.config.list_physical_devices('GPU'))

In [ ]:
def preprocess_audio_dataset(input_folder, target_sr=16000, target_length=80000, n_mels=40, n_fft=1024, hop_length=512, top_db=20):
    spectrograms = []
    labels = []

    # Iterate over each audio file in the input folder
    for filename in os.listdir(input_folder):

        if filename.endswith((".wav", ".mp3", ".webm")):

            file_path = os.path.join(input_folder, filename)
            # print(f"Processing {filename}")

            # Step 1: Load and resample audio
            y, sr = librosa.load(file_path, sr=target_sr)

            # Step 2: Trim or pad the audio to the target length (5 seconds)
            y = trim_or_pad_audio(y, target_length)

            # Step 3: Remove silence from the audio
            y = remove_silence(y, sr, top_db)

            # Step 4: Generate a log mel-spectrogram
            log_mel_spec = generate_mel_spectrogram(y, sr, n_mels, n_fft, hop_length)

            # Step 5: Resize the spectrogram to 432x288 pixels
            log_mel_spec_resized = resize_spectrogram(log_mel_spec)

            # Store the spectrogram and corresponding label 
            spectrograms.append(log_mel_spec_resized)
            labels.append(filename)
            
        # break
    return spectrograms, labels

def trim_or_pad_audio(y, target_length=80000):
    if len(y) > target_length:
        return y[:target_length]  # Trim to the target length
    else:
        return np.pad(y, (0, target_length - len(y)), 'constant')  # Pad with zeros

def remove_silence(y, sr, top_db=20):
    return librosa.effects.trim(y, top_db=top_db)[0]

def generate_mel_spectrogram(y, sr, n_mels=40, n_fft=1024, hop_length=512):
    mel_spec = librosa.feature.melspectrogram(
        y=y, 
        sr=sr, 
        n_mels=n_mels, 
        n_fft=n_fft, 
        hop_length=hop_length,
        fmin=20, 
        fmax=8000
    )
    log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
    return log_mel_spec

def resize_spectrogram(spec, target_size=(256, 375)): # (224, 224)
    # Resize the spectrogram to the desired size using OpenCV
    return cv2.resize(spec, target_size, interpolation=cv2.INTER_LINEAR)

def display_spectrogram(spectrogram, title='Spectrogram'):
    plt.figure(figsize=(10, 6))
    librosa.display.specshow(spectrogram, sr=16000, x_axis='time', y_axis='mel', fmin=20, fmax=8000, cmap='coolwarm')
    plt.colorbar(format='%+2.0f dB')
    plt.title(title)
    plt.tight_layout()
    plt.show()

# Normalize function based on ImageNet stats (mean and std)
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
    
def preprocess_spectrogram(spectrogram):
    # Convert spectrogram from shape (224, 224) to (224, 224, 3)
    
    # First, normalize the spectrogram if necessary (e.g., if pixel range is [0, 255])
    spectrogram = spectrogram.astype(np.float16) / 255.0
    
    # Replicate the spectrogram into 3 channels (to match ResNet50's expected input shape)
    spectrogram = np.repeat(spectrogram[..., np.newaxis], 3, axis=-1) # Adds 3rd dimension  3
   
    spectrogram = (spectrogram - mean) / std
    
    return spectrogram

In [ ]:
del true_labels,true_labels_bin,tpr

In [ ]:
# Preprocess the dataset from train folder
# train_folder = 'data/actual/train'
# test_folder = 'data/actual/test'

train_folder = 'data/first_two/train'
test_folder = 'data/first_two/test'

# train_folder = 'data/last_two/train'
# test_folder = 'data/last_two/test'


# Example for one language folder, say 'english' in the train folder
language_1 = 'punjabi' 
train_path_1 = os.path.join(train_folder, language_1)
#test_path_1 = os.path.join(test_folder, language_1)

language_2 = 'pushto' 
train_path_2 = os.path.join(train_folder, language_2)
#test_path_2 = os.path.join(test_folder, language_2)

language_3 = 'saraiki' 
train_path_3 = os.path.join(train_folder, language_3)
#test_path_3 = os.path.join(test_folder, language_3)

language_4 = 'sindhi' 
train_path_4 = os.path.join(train_folder, language_4)
#test_path_4 = os.path.join(test_folder, language_4)

language_5 = 'urdu'  
train_path_5 = os.path.join(train_folder, language_5)
#test_path_5 = os.path.join(test_folder, language_5)


# Preprocess the train dataset for the language
punjabi_spectrograms, punjabi_labels = preprocess_audio_dataset(train_path_1)
pushto_spectrograms, pushto_labels = preprocess_audio_dataset(train_path_2)
saraiki_spectrograms, saraiki_labels = preprocess_audio_dataset(train_path_3)
sindhi_spectrograms, sindhi_labels = preprocess_audio_dataset(train_path_4)
urdu_spectrograms, urdu_labels = preprocess_audio_dataset(train_path_5)

train_spectrograms = punjabi_spectrograms + sindhi_spectrograms + pushto_spectrograms + saraiki_spectrograms + urdu_spectrograms

train_labels = ['punjabi'] * len(punjabi_spectrograms) + \
         ['sindhi'] * len(sindhi_spectrograms) + \
         ['pushto'] * len(pushto_spectrograms) + \
         ['saraiki'] * len(saraiki_spectrograms) + \
         ['urdu'] * len(urdu_spectrograms)

assert len(train_spectrograms) == len(train_labels), "Mismatch between spectrograms and labels count."

print("Total Spectrograms: ", len(train_spectrograms))

# Display one spectrogram as a sample
# display_spectrogram(train_spectrograms[0], title="train Spectrogram")

In [ ]:
#del punjabi_spectrograms,sindhi_spectrograms,pushto_spectrograms,saraiki_spectrograms,urdu_spectrograms
del saraiki_labels,urdu_labels,punjabi_labels,sindhi_labels,pushto_labels
del sindhi_spectrograms,punjabi_spectrograms,pushto_spectrograms,urdu_spectrograms,saraiki_spectrograms
del train_folder,test_folder 
del train_path_1,train_path_2,train_path_3,train_path_4,train_path_5

In [ ]:
#print(f"Train Spectrograms Shape: {train_spectrograms.shape}")
# Preprocess training and testing spectrograms
train_spectrograms = np.array([preprocess_spectrogram(s) for s in train_spectrograms])
#test_spectrograms = np.array([preprocess_spectrogram(s) for s in test_spectrograms])

# Ensure spectrograms are the correct shape
print(f"Train Spectrograms Shape: {train_spectrograms.shape}")
#print(f"Test Spectrograms Shape: {test_spectrograms.shape}")


In [ ]:
# Ensure spectrograms are the correct shape
print(f"Train Spectrograms Shape: {train_spectrograms.shape}")

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


# Encode labels using LabelEncoder
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)  # Fit and transform the training labels

# Print label mapping for reference
print("Label Mapping:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

train_spectrograms, val_spectrograms, train_labels_encoded, val_labels_encoded = train_test_split(
    train_spectrograms, train_labels_encoded, test_size=0.2, random_state=42
)

# 1. Create TensorFlow Dataset objects for train and test datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_spectrograms, train_labels_encoded))
val_dataset = tf.data.Dataset.from_tensor_slices((val_spectrograms, val_labels_encoded))


# Shuffle and batch the train dataset
train_dataset = train_dataset.shuffle(len(train_spectrograms)).batch(16)

# Batch the validation and test datasets (no shuffle needed)
val_dataset = val_dataset.batch(16)

# 4. Inspect the dataset (first batch example)
for images, labels in train_dataset.take(1):  # Just take 1 batch to inspect
    print("Batch image shape:", images.shape)
    print("Batch labels shape:", labels.shape)

# Now you can feed these datasets to your model during training
# Example:
# model.fit(train_dataset, validation_data=test_dataset, epochs=10)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the CNN model
def create_cnn_model(input_shape, num_classes):
    model = models.Sequential([
        layers.InputLayer(input_shape=(375, 256, 3)),
        layers.Normalization(axis=-1),
        layers.Conv2D(16, (7, 5), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((3, 2)),
        layers.Conv2D(32, (5, 5), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.2),
        layers.Dense(5, activation='softmax')  # Output layer for classification
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Input shape of spectrograms (224, 224, 3)
input_shape = (375, 256, 3)
num_classes = 5  # 5 languages

# Create the CNN model
model = create_cnn_model(input_shape, num_classes)

# Summary of the model
model.summary()
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=7,  # Stop after 2 epochs without improvement
    restore_best_weights=True
    )
lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
class BatchLossLogger(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.batch_losses = []
        
    def on_batch_end(self, batch, logs=None):
        self.batch_losses.append(logs['loss'])

batch_logger1 = BatchLossLogger()
checkpoint = ModelCheckpoint(
filepath='best_model_pt1.keras',  # Save file path
monitor='val_loss',       # Metric to monitor (e.g., 'val_loss' or 'val_accuracy')
save_best_only=True,      # Save only the model with the best performance
mode='min',               # Mode for monitoring ('min' for loss, 'max' for accuracy)
verbose=1                 # Verbosity level (1 to print saving logs)
)

# Train the model
history1=model.fit(
train_dataset,  # Replace with the 4th folder's training dataset
validation_data=val_dataset,  # Replace with the 4th folder's validation dataset
epochs=20,  # Reduced maximum number of epochs
callbacks=[lr_schedule,early_stopping,checkpoint,batch_logger1
],
verbose=2
)

In [ ]:
# del train_labels,train_labels_encoded,train_spectrograms
# del val_labels_encoded,val_spectrograms
# del early_stopping,checkpoint,images,input_shape,labels,num_classes
import gc
gc.collect()
#use traindataset and val dataset to train next model

In [ ]:
# Preprocess the dataset from train folder
# train_folder = 'data/actual/train'
# test_folder = 'data/actual/test'

train_folder_2 = 'data/last_two/train'
test_folder_2 = 'data/last_two/test'

# Example for one language folder, say 'english' in the train folder
#language_1_2 = 'punjabi' 
train_path_12 = os.path.join(train_folder_2, language_1)
#test_path_1 = os.path.join(test_folder, language_1)

#language_2 = 'pushto' 
train_path_22 = os.path.join(train_folder_2, language_2)
#test_path_2 = os.path.join(test_folder, language_2)

#language_3 = 'saraiki' 
train_path_32 = os.path.join(train_folder_2, language_3)
#test_path_3 = os.path.join(test_folder, language_3)

#language_4 = 'sindhi' 
train_path_42 = os.path.join(train_folder_2, language_4)
#test_path_4 = os.path.join(test_folder, language_4)

#language_5 = 'urdu'  
train_path_52 = os.path.join(train_folder_2, language_5)
#test_path_5 = os.path.join(test_folder, language_5)


# Preprocess the train dataset for the language
punjabi_spectrograms_2, punjabi_labels = preprocess_audio_dataset(train_path_12)
pushto_spectrograms_2, pushto_labels = preprocess_audio_dataset(train_path_22)
saraiki_spectrograms_2, saraiki_labels = preprocess_audio_dataset(train_path_32)
sindhi_spectrograms_2, sindhi_labels = preprocess_audio_dataset(train_path_42)
urdu_spectrograms_2, urdu_labels = preprocess_audio_dataset(train_path_52)

train_spectrograms_2 = punjabi_spectrograms_2 + sindhi_spectrograms_2 + pushto_spectrograms_2 + saraiki_spectrograms_2 + urdu_spectrograms_2

train_labels_2 = ['punjabi'] * len(punjabi_spectrograms_2) + \
         ['sindhi'] * len(sindhi_spectrograms_2) + \
         ['pushto'] * len(pushto_spectrograms_2) + \
         ['saraiki'] * len(saraiki_spectrograms_2) + \
         ['urdu'] * len(urdu_spectrograms_2)

assert len(train_spectrograms_2) == len(train_labels_2), "Mismatch between spectrograms and labels count."

print("Total Spectrograms: ", len(train_spectrograms_2))
train_spectrograms_2 = np.array([preprocess_spectrogram(s) for s in train_spectrograms_2])
#test_spectrograms = np.array([preprocess_spectrogram(s) for s in test_spectrograms])

# Ensure spectrograms are the correct shape
print(f"Train Spectrograms Shape: {train_spectrograms_2.shape}")
# Display one spectrogram as a sample
# display_spectrogram(train_spectrograms[0], title="train Spectrogram")

In [ ]:
#del punjabi_spectrograms,sindhi_spectrograms,pushto_spectrograms,saraiki_spectrograms,urdu_spectrograms
import gc
del saraiki_labels,urdu_labels,punjabi_labels,sindhi_labels,pushto_labels
del sindhi_spectrograms_2,punjabi_spectrograms_2,pushto_spectrograms_2,urdu_spectrograms_2,saraiki_spectrograms_2
del train_folder_2,test_folder_2 
del train_path_12,train_path_22,train_path_32,train_path_42,train_path_52
gc.collect()

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split



# Encode labels using LabelEncoder
#label_encoder = LabelEncoder()
train_labels_encoded_2 = label_encoder.fit_transform(train_labels_2)  # Fit and transform the training labels

# Print label mapping for reference
print("Label Mapping:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

train_spectrograms_2, val_spectrograms_2, train_labels_encoded_2, val_labels_encoded_2 = train_test_split(
    train_spectrograms_2, train_labels_encoded_2, test_size=0.2, random_state=42
)

# 1. Create TensorFlow Dataset objects for train and test datasets
train_dataset_2 = tf.data.Dataset.from_tensor_slices((train_spectrograms_2, train_labels_encoded_2))
val_dataset_2 = tf.data.Dataset.from_tensor_slices((val_spectrograms_2, val_labels_encoded_2))


# Shuffle and batch the train dataset
train_dataset_2 = train_dataset_2.shuffle(len(train_spectrograms_2)).batch(16)

# Batch the validation and test datasets (no shuffle needed)
val_dataset_2 = val_dataset_2.batch(16)

# 4. Inspect the dataset (first batch example)
for images, labels in train_dataset_2.take(1):  # Just take 1 batch to inspect
    print("Batch image shape:", images.shape)
    print("Batch labels shape:", labels.shape)

# Now you can feed these datasets to your model during training
# Example:
# model.fit(train_dataset, validation_data=test_dataset, epochs=10)


In [ ]:
del train_labels_2,train_labels_encoded_2,train_spectrograms_2
del val_labels_encoded_2,val_spectrograms_2
del images,labels

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
#model = load_model('cnn_pt1.keras')
model = load_model('best_model_pt1.keras')

# Load the model trained on the 3rd folder

early_stopping_2 = EarlyStopping(
    monitor='val_loss',
    patience=7,  # Stop after 2 epochs without improvement
    restore_best_weights=True
    )

    

lr_schedule_2 = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
batch_logger_2 = BatchLossLogger()
checkpoint_2 = ModelCheckpoint(
filepath='best_model_pt2.keras',  # Save file path
monitor='val_loss',       # Metric to monitor (e.g., 'val_loss' or 'val_accuracy')
save_best_only=True,      # Save only the model with the best performance
mode='min',               # Mode for monitoring ('min' for loss, 'max' for accuracy)
verbose=1                 # Verbosity level (1 to print saving logs)
)
# Train the model
# Train the model
model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
history2=model.fit(
train_dataset_2,  # Replace with the 4th folder's training dataset
validation_data=val_dataset_2,  # Replace with the 4th folder's validation dataset
epochs=20,  # Reduced maximum number of epochs
callbacks=[lr_schedule_2,early_stopping_2,checkpoint_2,batch_logger_2
],
verbose=2
)


In [ ]:
combined_loss = history1.history['loss'] + history2.history['loss']
combined_val_loss = history1.history['val_loss'] + history2.history['val_loss']
#96 train acc 80 valid acc for best model pt1 : its bestmodelpt2
#97 train acc 71 valid acc cnnpt2

In [ ]:
import matplotlib.pyplot as plt

epochs = list(range(1, len(combined_loss) + 1))

plt.figure(figsize=(10, 6))
plt.plot(epochs, combined_loss, label='Training Loss')
plt.plot(epochs, combined_val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs. Epochs')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Combine batch losses
combined_batch_losses = batch_logger1.batch_losses + batch_logger_2.batch_losses
steps = list(range(1, len(combined_batch_losses) + 1))

plt.figure(figsize=(10, 6))
plt.plot(steps, combined_batch_losses, label='Loss Per Step')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Loss vs. Steps')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
del val_dataset,val_dataset_2,train_dataset,train_dataset_2,train_labels,train_labels_encoded,train_spectrograms,

In [ ]:
# Preprocess the dataset from train folder

test_folder = 'data/first_two/test'
test_folder_2 = 'data/last_two/test'

# Example for one language folder, say 'english' in the train folder
language_1 = 'punjabi' 

test_path_1 = os.path.join(test_folder, language_1)
test_path_12 = os.path.join(test_folder_2, language_1)


language_2 = 'pushto' 

test_path_2 = os.path.join(test_folder, language_2)
test_path_22 = os.path.join(test_folder_2, language_2)

language_3 = 'saraiki' 

test_path_3 = os.path.join(test_folder, language_3)
test_path_32 = os.path.join(test_folder_2, language_3)

language_4 = 'sindhi' 

test_path_4 = os.path.join(test_folder, language_4)
test_path_42 = os.path.join(test_folder_2, language_4)

language_5 = 'urdu'  

test_path_5 = os.path.join(test_folder, language_5)
test_path_52 = os.path.join(test_folder_2, language_5)

# Preprocess the test dataset for the language
test_punjabi_spectrograms, test_punjabi_labels = preprocess_audio_dataset(test_path_1)
test_pushto_spectrograms, test_pushto_labels = preprocess_audio_dataset(test_path_2)
test_saraiki_spectrograms, test_saraiki_labels = preprocess_audio_dataset(test_path_3)
test_sindhi_spectrograms, test_sindhi_labels = preprocess_audio_dataset(test_path_4)
test_urdu_spectrograms, test_urdu_labels = preprocess_audio_dataset(test_path_5)

test2_punjabi_spectrograms, test2_punjabi_labels = preprocess_audio_dataset(test_path_12)
test2_pushto_spectrograms, test2_pushto_labels = preprocess_audio_dataset(test_path_22)
test2_saraiki_spectrograms, test2_saraiki_labels = preprocess_audio_dataset(test_path_32)
test2_sindhi_spectrograms, test2_sindhi_labels = preprocess_audio_dataset(test_path_42)
test2_urdu_spectrograms, test2_urdu_labels = preprocess_audio_dataset(test_path_52)


test_spectrograms = test_punjabi_spectrograms+ test2_punjabi_spectrograms+ test_sindhi_spectrograms+test2_sindhi_spectrograms + test_pushto_spectrograms +test2_pushto_spectrograms+ test_saraiki_spectrograms+test2_saraiki_spectrograms  + test_urdu_spectrograms+test2_urdu_spectrograms 

test_labels = [0] * len(test_punjabi_spectrograms+test2_punjabi_spectrograms) + \
         [3] * len(test_sindhi_spectrograms+test2_sindhi_spectrograms) + \
         [1] * len(test_pushto_spectrograms+test2_pushto_spectrograms) + \
         [2] * len(test_saraiki_spectrograms+test2_saraiki_spectrograms ) + \
         [4] * len(test_urdu_spectrograms+test2_urdu_spectrograms )
assert len(test_spectrograms) == len(test_labels), "Mismatch between spectrograms and labels count."

print("Total Spectrograms: ", len(test_spectrograms))
print("Total Labels: ", len(test_labels))
test_spectrograms = np.array([preprocess_spectrogram(s) for s in test_spectrograms])

# Ensure spectrograms are the correct shape

print(f"Test Spectrograms Shape: {test_spectrograms.shape}")

In [ ]:
# del language_1,language_2,language_3,language_4,language_5
# del test_folder,test_folder_2
# del test_path_1,test_path_12,test2_urdu_spectrograms,test2_punjabi_labels,test2_pushto_labels
# del test_path_2,test_path_22,test_path_3,test_path_32,test_path_4,test_path_42,test_path_5,test_path_52
# del test_punjabi_labels,test_punjabi_spectrograms,test_pushto_labels,test_pushto_spectrograms,test_saraiki_labels,test_saraiki_spectrograms
# del test_sindhi_labels,test_sindhi_spectrograms,test_urdu_labels,test_urdu_spectrograms
# del test2_punjabi_spectrograms,test2_pushto_spectrograms,test2_saraiki_labels,test2_saraiki_spectrograms,test2_sindhi_labels
# del test2_sindhi_spectrograms,test2_urdu_labels
# del batch_logger1,batch_logger_2    
# del combined_batch_losses,combined_loss,combined_val_loss
# del epochs,history1,history2,language_1
del language_2,language_3,language_4,language_5,steps



In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

def evaluate_entire_testset(model, test_spectrograms, test_labels, labels_class):
    """
    Evaluates the model on the entire test set and computes the accuracy, confusion matrix,
    and a classification report.

    Args:
        model: The trained model to evaluate.
        test_spectrograms: Array of test spectrograms.
        test_labels: Array of corresponding test labels.
        labels_class: List of all possible class labels (e.g., [0, 1, 2, 3, 4]).

    Returns:
        overall_accuracy: The accuracy of the model on the test set.
        cm: The confusion matrix for the test set.
    """
    # Convert test data into a TensorFlow Dataset
    test_dataset = tf.data.Dataset.from_tensor_slices((test_spectrograms, test_labels))
    test_dataset = test_dataset.batch(32)  # Batch size

    # Predictions and true labels
    predictions = []
    true_labels = []

    for images, labels in test_dataset:
        # Predict the labels
        preds = model.predict(images)
        predicted_labels = np.argmax(preds, axis=1)  # Get the predicted class
        predictions.extend(predicted_labels)
        true_labels.extend(labels.numpy())  # Convert true labels to numpy array

    # Compute overall accuracy
    overall_accuracy = accuracy_score(true_labels, predictions)
    print(f"Overall Accuracy: {overall_accuracy:.4f}")

    # Compute the confusion matrix
    cm = confusion_matrix(true_labels, predictions, labels=labels_class)

    # Display the confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels_class, yticklabels=labels_class)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix for Entire Test Set')
    plt.show()

    # Print the classification report
    report = classification_report(true_labels, predictions, target_names=[f'Class {label}' for label in labels_class])
    print("Classification Report:")
    print(report)

    return overall_accuracy, cm


# Example usage
from tensorflow.keras.models import load_model
model = load_model('best_model_pt2.keras')  # Load your model

# Assuming `test_spectrograms` and `test_labels` are already defined
labels_class = [0, 1, 2, 3, 4]  # All possible classes
overall_accuracy, cm = evaluate_entire_testset(model, test_spectrograms, test_labels, labels_class)


In [ ]:
import numpy as np
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import tensorflow as tf
import matplotlib.pyplot as plt

# Assuming test_dataset is your tf.data.Dataset, and the model is already trained
test_dataset = tf.data.Dataset.from_tensor_slices((test_spectrograms, test_labels))
test_dataset = test_dataset.batch(16)  
# Initialize lists to store the true labels and predicted probabilities
true_labels = []
pred_probs = []

# Iterate over the test dataset
for spectrograms, labels in test_dataset:
    # Predict probabilities for the batch
    batch_pred_probs = model.predict(spectrograms)
    
    # Store the true labels and predicted probabilities
    true_labels.append(labels.numpy())  # Convert Tensor to numpy array
    pred_probs.append(batch_pred_probs)

# Convert lists to numpy arrays
true_labels = np.concatenate(true_labels, axis=0)
pred_probs = np.concatenate(pred_probs, axis=0)

# Binarize the true labels for ROC
true_labels_bin = label_binarize(true_labels, classes=[0, 1, 2, 3, 4])
n_classes = true_labels_bin.shape[1]

# Plot ROC curve
plt.figure(figsize=(10, 8))

for i in range(n_classes):
    fpr, tpr, _ = roc_curve(true_labels_bin[:, i], pred_probs[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"Class {i} (AUC = {roc_auc:.2f})")

# Diagonal line (random classifier)
plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line

# Add labels and title
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Test Dataset')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()


In [ ]:
del images,input_shape,label_encoder,labels,labels_class,model,n_classes,num_classes
